In [1]:
from preprocess import *
tensor,sr = load_mp3_files("../dataset")

for i in tensor:
    print(f"tensor{i}.shape: {i.shape}")
print(f"Sampling rate: {sr}")


tensortensor([[0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 4.0139e-05, 3.4890e-05,
         3.2205e-05]]).shape: torch.Size([1, 4333568])
tensortensor([[ 1.1999e-07, -1.1272e-08, -5.3291e-07,  ...,  2.7773e-11,
         -1.3056e-11,  2.6297e-12]]).shape: torch.Size([1, 2189056])
tensortensor([[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ..., -4.3125e-05,
         -4.0388e-05, -3.3079e-05]]).shape: torch.Size([1, 2569472])
tensortensor([[6.1613e-03, 1.7957e-02, 2.6801e-02,  ..., 2.1595e-05, 2.2561e-05,
         1.6976e-05]]).shape: torch.Size([1, 1050880])
tensortensor([[0.0000, 0.0000, 0.0000,  ..., 0.0005, 0.0005, 0.0005]]).shape: torch.Size([1, 2685440])
tensortensor([[0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 1.7031e-05, 1.4117e-05,
         7.4276e-06]]).shape: torch.Size([1, 3948032])
tensortensor([[ 5.7710e-12,  9.9875e-12, -5.6306e-12,  ...,  7.1811e-12,
         -3.2290e-12, -1.1336e-11]]).shape: torch.Size([1, 2854912])
tensortensor([[ 5.7710e-12,  9.9875e-12, -5.6306e-12,  ...,  3.

In [5]:
# make them into batched x,y. 
tensor_stack = torch.cat(tensor,dim=-1)
print(f"tensor_stack.shape: {tensor_stack.shape}")

tensor_stack.shape: torch.Size([1, 33978368])


In [21]:
ck_len = 512*600 # for first 32 seconds, we predict the next 32 seconds (sampling rate = 8000)

chunks = create_overlapping_chunks_tensor(tensor_stack,chunk_len=ck_len)
print(chunks.shape) #torch.Size([706, 96000])
x,y = chunks[:,:ck_len//2], chunks[:,ck_len//2:]
print(f"x: {x.shape}")
print(f"y: {y.shape}")




torch.Size([6629, 40960])
x: torch.Size([6629, 20480])
y: torch.Size([6629, 20480])


In [22]:

indices = torch.randperm(x.size(0))

shuffled_x,shuffled_y = x[indices],y[indices]

dSet = {
    'x': shuffled_x[:3450,:],
    'y': shuffled_y[:3450,:],
    'x_test': shuffled_x[3450:,:],
    'y_test': shuffled_y[3450:,:],
}
from torch.utils.data import TensorDataset,DataLoader
trainDataset,testDataset = TensorDataset(dSet['x'],dSet['y']),TensorDataset(dSet['x_test'],dSet['y_test'])
dLoader,dLoader_test = DataLoader(trainDataset,batch_size=1,shuffle=True),DataLoader(testDataset,batch_size=1,shuffle=False)

In [23]:

from layers.main_model import net
LR=1e-4
device = torch.device('cuda:0')

model = net(sequence_length=512*300,num_blocks=6,activation='silu').to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(),lr=LR,)
num_epochs = 500
train_losses = []
test_losses = []



In [24]:

import wandb
from datetime import datetime
current_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

wandb.init(project="audio-gen", config={
    "epochs": num_epochs,
    "batch_size": len(dLoader),  # assuming dLoader gives one batch per step
    "learning_rate": LR,
    "device": "cuda RTX 3080 Ti",
    "ck_len": ck_len,
    "num_blocks": 6,
    },
    name=f"run_{current_time}"
)

for epoch in range(num_epochs):
    model.train()  # Set model to training mode
    running_loss = 0.0
    for i,(inputs, labels) in enumerate(dLoader):
        
        # Zero the parameter gradients
        if torch.any(torch.isnan(inputs)) or torch.any(torch.isnan(labels)):
            print("Input or labels contain NaN values.")
            
        inputs = inputs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        # Backward pass and optimization
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=10.0)
        optimizer.step()
        wandb.log({'single_loss': loss.item()})
        # Accumulate loss
        running_loss += loss.item()
    
    # Average loss for the epoch
    epoch_loss = running_loss / len(dLoader)
    train_losses.append(epoch_loss)
    wandb.log({"epoch_train_loss": epoch_loss})
    print(f'Epoch [{epoch+1}/{num_epochs}], Training Loss: {epoch_loss:.4f}')
    
    
    if epoch % 5 == 0:
        torch.save(model.state_dict(), f'models/modelDict_epoch_{epoch+1}.pth')
        torch.save(model, f'models/model_epoch_{epoch+1}.pth')
        model.eval() 
        running_loss_test = 0.0

        with torch.no_grad(): 
            for inputs, labels in dLoader_test:
                inputs = inputs.to(device)
                labels = labels.to(device)
                outputs = model(inputs)
                for name, param in model.named_parameters():
                    if param.grad is not None and torch.any(torch.isnan(param.grad)):
                        print(f"Gradient for {name} contains NaN values.")
                        
                loss = criterion(outputs, labels)
                running_loss_test += loss.item()
         
        
        
        
        
        # Average validation loss for the epoch
        epoch_test_loss = running_loss_test / len(dLoader_test)
        test_losses.append(epoch_test_loss)
        wandb.log({"eval_loss":epoch_test_loss})    

        print(f'Epoch [{epoch+1}/{num_epochs}], Validation Loss: {epoch_test_loss:.4f}')
    
# At the end, you can plot the losses if needed

loss: 8.490424806950614e-05
loss: 0.002055252203717828
loss: 0.003938093315809965
loss: 0.006290995515882969
loss: 0.001245544059202075
loss: 0.0005569446948356926
loss: 0.00013573670003097504
loss: 0.0008337479084730148
loss: 0.0019607783760875463
loss: 0.006679121404886246
loss: 0.0008255488937720656
loss: 0.0002555732789915055
loss: 0.00013253148063085973
loss: 0.004887404385954142
loss: 0.003138697938993573
loss: 0.0011610668152570724
loss: 0.004229875281453133
loss: 2.242262962681707e-05
loss: 0.008252294734120369
loss: 0.001216390635818243
loss: 0.002600605133920908
loss: 0.0013715634122490883
loss: 0.00019980059005320072
loss: 8.914861973607913e-05
loss: 0.005425921641290188
loss: 0.00017995423695538193
loss: 0.005412047263234854
loss: 0.005595244932919741
loss: 5.37126325070858e-05
loss: 0.0005044179852120578
loss: 0.00019768928177654743
loss: 0.00032757720327936113
loss: 0.00029478437500074506
loss: 0.0017181731527671218
loss: 0.000912757299374789
loss: 9.438435517949983e-05
l

Thread SenderThread:
Traceback (most recent call last):
  File "/opt/homebrew/lib/python3.11/site-packages/wandb/sdk/internal/internal_util.py", line 48, in run
    self._run()
  File "/opt/homebrew/lib/python3.11/site-packages/wandb/sdk/internal/internal_util.py", line 99, in _run
    self._process(record)
  File "/opt/homebrew/lib/python3.11/site-packages/wandb/sdk/internal/internal.py", line 327, in _process
    self._sm.send(record)
  File "/opt/homebrew/lib/python3.11/site-packages/wandb/sdk/internal/sender.py", line 398, in send
    send_handler(record)
  File "/opt/homebrew/lib/python3.11/site-packages/wandb/sdk/internal/sender.py", line 420, in send_request
    send_handler(record)
  File "/opt/homebrew/lib/python3.11/site-packages/wandb/sdk/internal/sender.py", line 1224, in send_request_summary_record
    self._update_summary_record(record.request.summary_record.summary)
  File "/opt/homebrew/lib/python3.11/site-packages/wandb/sdk/internal/sender.py", line 1218, in _update_su

BrokenPipeError: [Errno 32] Broken pipe

In [ ]:
import torch
import torch.nn as nn
import torchaudio
import os

from layers.main_model import net

device = torch.device('cpu')
model = torch.load("model_epoch_45.pth")
criterion = nn.MSELoss()

# Assuming dLoader_test is your test dataloader and each input/label has shape (1, L)
output_folder = "eval_audio"  # Folder to store audio files
os.makedirs(output_folder, exist_ok=True)  # Create folder if it doesn't exist

# Evaluation mode
model.eval() 

with torch.no_grad(): 
    for batch_idx, (inputs, labels) in enumerate(dLoader_test):
        inputs = inputs.to(device)  # Shape (1, L)
        labels = labels.to(device)  # Shape (1, L)

        # Forward pass
        outputs = model(inputs)  # Shape (1, L)

        # Concatenate inputs, labels, and outputs
        concatenated_audio = torch.cat((inputs, outputs,labels), dim=1)  # Shape (1, 3*L)

        # Convert to CPU and detach (if necessary)
        concatenated_audio = concatenated_audio.cpu().detach()

        # Save as audio (normalize to [-1, 1] if needed)
        file_name = f"{output_folder}/audio_batch_{batch_idx}.wav"
        torchaudio.save(file_name, concatenated_audio, sample_rate=SR)

        print(f"Saved concatenated audio for batch {batch_idx} to {file_name}")


In [ ]:
import torch
from layers.main_model import net
def print_model_info(model):
    # Calculate total number of parameters
    total_params = sum(p.numel() for p in model.parameters())
    
    # Assuming parameters are stored as float32 (4 bytes per parameter)
    param_size_bytes = total_params * 4  # 4 bytes for float32
    param_size_gb = param_size_bytes / (1024 ** 3)  # Convert to GB
    
    print(f"Total number of parameters: {total_params:,}")
    print(f"Model size: {param_size_gb:.2f} GB")

# Example usage
model = net(sequence_length=8000*30, num_blocks=12, activation='silu',lstm_option=True)
print_model_info(model)
